# 재구축 데이터셋 Scratch



In [2]:
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset
from PIL import Image
import numpy as np
from tqdm import tqdm

In [7]:
# 구축된 .npy파일을 Pytorch DataLoader을 사용할 수 있도록 CUSTOM DATASET을 만듬.
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split

default_path = "/content/drive/My Drive/Colab Notebooks/final/"

CUB200_TYPE_TRAIN = 1
CUB200_TYPE_TEST = 2
CUB200_TYPE_SUBMIT = 3

drive.mount('/content/drive')
class CUB200(data.Dataset):

    def __init__(self, type, transform = None):
        super(CUB200, self).__init__()
        """
        type : int = 1, 2, 3
        """

        original_train_data = np.load(default_path + 'train_image.npy')
        original_train_label = np.load(default_path + 'train_label.npy')

        train_data, test_data, train_label, test_label = train_test_split(
            original_train_data,
            original_train_label,
            test_size = 0.3,
            random_state = 1)
        
        if type == CUB200_TYPE_TRAIN:
          self.image = train_data
          self.label = train_label
        elif type == CUB200_TYPE_TEST:
          self.image = test_data
          self.label = test_label
        elif type == CUB200_TYPE_SUBMIT:
          self.image = np.load(default_path + 'test_image.npy')
          self.label = None
        
        self.transform = transform

    def __getitem__(self, index):
        img, target = self.image[index], self.label[index]
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        return img, target

    def __len__(self):
        return len(self.image)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
trainCUB = CUB200(CUB200_TYPE_TRAIN)
print(trainCUB.image.shape)
print(trainCUB.label.shape)

testCUB = CUB200(CUB200_TYPE_TEST)
print(testCUB.image.shape)
print(testCUB.label.shape)
print(np.max(testCUB.label), np.min(testCUB.label))

submitCUB = CUB200(CUB200_TYPE_SUBMIT)
print(submitCUB.image.shape)

(626, 256, 256, 3)
(626,)
(269, 256, 256, 3)
(269,)
49 0
(500, 256, 256, 3)


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [31]:
# train_data에만 data augmentaion을 적용
transform_train = transforms.Compose([
        transforms.RandomCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [32]:
# CUSTOM DATASET을 이용하여 train_loader, test_loader을 구축

batch_size = 8

train_loader = torch.utils.data.DataLoader(
    dataset = CUB200(CUB200_TYPE_TRAIN, transform = transform_train),
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = CUB200(CUB200_TYPE_TEST, transform = transform_test),
    batch_size = batch_size,
    shuffle = False
)

In [33]:
def training_model(model, criterion, optimizer, scheduler, num_epochs = 25):

    for epoch in range(num_epochs):
        scheduler.step()

        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 60 == 59:
                print('[%d, %5d] loss: %.7f' %
                    (epoch + 1, (i + 1), running_loss / 20))
                running_loss = 0.0
        
        train_correct = 0
        train_total = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs = inputs.squeeze()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        print('[%d epoch] Accuracy of the network on the train images: %d %%' %
              (epoch + 1, 100 * train_correct / train_total))
        
    print("End Training do it eval_accuracy")
    return model

In [34]:
def eval_accuracy(model):
    class_correct = list(0. for i in range(50))
    class_total = list(0. for i in range(50))

    correct = 0
    total = 0
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            print("#", i, outputs)
            c = (predicted == labels).squeeze()
                    
            for i in range(labels.shape[0]):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1
                
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

    print('Accuracy of the network on test images: %d %%' % (
        100 * correct / total))            
                
    return 

In [71]:
model_ft = models.resnet152(pretrained=True)
for param in model_ft.parameters():
  param.requires_grad = False
model_ft.fc = nn.Sequential(
        nn.Linear(model_ft.fc.in_features, 512),
        nn.Linear(512, 256),
        nn.Linear(256, 50),
        # nn.Softmax(),
    )
model_ft

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/hub/checkpoints/resnet152-b121ed2d.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [74]:
num_epochs = 25
model_ft.to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model_ft.parameters(), lr = 0.001)
optimizer = optim.SGD(model_ft.parameters(), lr=0.005, momentum=0.9)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

model_ft = training_model(model_ft, criterion, optimizer, lr_scheduler, num_epochs)

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,    60] loss: 6.8866126
[1 epoch] Accuracy of the network on the train images: 60 %
[2,    60] loss: 4.1696811
[2 epoch] Accuracy of the network on the train images: 69 %
[3,    60] loss: 2.9270913
[3 epoch] Accuracy of the network on the train images: 83 %
[4,    60] loss: 1.7830543
[4 epoch] Accuracy of the network on the train images: 83 %
[5,    60] loss: 1.4756389
[5 epoch] Accuracy of the network on the train images: 92 %
[6,    60] loss: 0.9382226
[6 epoch] Accuracy of the network on the train images: 96 %
[7,    60] loss: 0.4551730
[7 epoch] Accuracy of the network on the train images: 98 %
[8,    60] loss: 0.3524709
[8 epoch] Accuracy of the network on the train images: 98 %
[9,    60] loss: 0.3464046
[9 epoch] Accuracy of the network on the train images: 98 %
[10,    60] loss: 0.2965732
[10 epoch] Accuracy of the network on the train images: 99 %
[11,    60] loss: 0.2735408
[11 epoch] Accuracy of the network on the train images: 99 %
[12,    60] loss: 0.2808554
[12 epoch] 

In [75]:
eval_accuracy(model_ft)

# 0 tensor([[-1.4371e-01, -1.7514e+00, -2.3483e+00, -5.1818e+00,  1.6239e+00,
          1.0145e+00, -7.8045e-01,  4.8713e+00, -1.4229e+00,  4.3067e+00,
         -1.9990e+00, -4.0935e+00,  2.8731e-01, -2.3447e+00,  2.0721e+00,
         -5.3589e+00, -5.7674e+00, -6.1586e+00, -2.2109e+00,  1.0991e+01,
         -5.4549e-02, -3.4555e+00,  5.4401e+00,  3.7297e+00, -1.9855e-01,
          1.4783e+00,  3.2234e-01, -1.8400e+00, -6.9594e+00, -9.0206e-01,
         -2.9677e+00, -1.2932e+00, -3.6293e+00, -9.6434e-01, -1.9377e+00,
         -4.1121e+00, -9.7153e-01,  4.0800e+00, -3.3239e+00,  4.8406e+00,
          5.6753e+00,  1.3822e+00,  7.2359e+00, -5.3139e-01, -1.7510e+00,
          3.2618e+00, -1.4198e+00,  5.0310e+00,  6.8446e-01,  1.9244e+00],
        [ 4.9388e+00,  5.8340e+00,  2.5602e+00,  3.0038e+00,  4.0217e+00,
          2.0396e+00,  8.3786e-01,  3.9229e+00,  3.4861e+00, -4.4330e-01,
         -4.6358e-01,  1.6265e+00, -5.2810e-01, -4.4307e-01, -1.2774e+00,
          2.7185e+00,  1.3802e+00